In [3]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import re
import flair

In [4]:
def cleanText(text):
    whitespace = re.compile(r"\s+")
    web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
    user = re.compile(r"(?i)@[a-z0-9_]+")
    text = whitespace.sub(' ', text)
    text = web_address.sub('', text)
    text = user.sub('', text)
    text = re.sub(r"(?:@\S*|#\S*|http(?=.*://)\S*)", "", text)
    return text

In [15]:
dataFrame = pd.read_csv('./data/daily.csv')
dataFrame = dataFrame[4507:]
dataFrame.head()

,Day,Price
4507,20150105,3.22
4508,20150106,2.98
4509,20150107,3.08
4510,20150108,2.92
4511,20150109,2.96


In [10]:
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

2022-05-02 20:24:38,560 loading file C:\Users\007ra\.flair\models\sentiment-en-mix-distillbert_4.pt


In [16]:
data = {}
MAX_TWEETS = 50


for n in range (1, len(dataFrame['Day'])):
    PREV = str(list(dataFrame.Day)[n-1])[:4]+'-'+str(list(dataFrame.Day)[n-1])[4:6]+'-'+str(list(dataFrame.Day)[n-1])[6:]
    PRES = str(list(dataFrame.Day)[n])[:4]+'-'+str(list(dataFrame.Day)[n])[4:6]+'-'+str(list(dataFrame.Day)[n])[6:]
    QUERY = f"natural gas (natural OR gas OR import OR export OR price) until:{PRES} since:{PREV} -filter:links -filter:replies"
    print(f'checking for {PRES}')
    curr = 0.0
    nums = 0
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(QUERY).get_items()):
        nums += 1
        if i>MAX_TWEETS:
            break
        sentence = flair.data.Sentence(cleanText(tweet.content))
        sentiment_model.predict(sentence)
        try:
            if(sentence.labels[0].value == 'POSITIVE'):
                curr += sentence.labels[0].score
            else:
                curr -= sentence.labels[0].score
        except:
            pass
    dataFrame.loc[dataFrame.Price == list(dataFrame['Price'])[n], 'Sentiment'] = curr/nums
    data[PRES] = curr/nums

checking for 2015-01-06
checking for 2015-01-07
checking for 2015-01-08
checking for 2015-01-09
checking for 2015-01-12
checking for 2015-01-13
checking for 2015-01-14
checking for 2015-01-15
checking for 2015-01-16
checking for 2015-01-20
checking for 2015-01-21
checking for 2015-01-22
checking for 2015-01-23
checking for 2015-01-26
checking for 2015-01-27
checking for 2015-01-28
checking for 2015-01-29
checking for 2015-01-30
checking for 2015-02-02
checking for 2015-02-03
checking for 2015-02-04
checking for 2015-02-05
checking for 2015-02-06
checking for 2015-02-09
checking for 2015-02-10
checking for 2015-02-11
checking for 2015-02-12
checking for 2015-02-13
checking for 2015-02-17
checking for 2015-02-18
checking for 2015-02-19
checking for 2015-02-20
checking for 2015-02-23
checking for 2015-02-24
checking for 2015-02-25
checking for 2015-02-26
checking for 2015-02-27
checking for 2015-03-02
checking for 2015-03-03
checking for 2015-03-04
checking for 2015-03-05
checking for 201

KeyboardInterrupt: 

In [19]:
dataFrame.to_csv('./data/dailySentiment.csv')